from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 400em; }</style>"))
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
# import
import datetime
from datetime import timedelta, date
from datetime import datetime

def cast_date(dt):
    """
    casting function
    :return: return date i format of date
    """
    date_str = datetime.strptime(dt, "%d.%m.%Y").strftime("%Y-%m-%d")
    return date(*map(int, date_str.split('-')))


def all_ticket_options():
    """
    function receives input from the user and checks for correctness and adds to the dates 4 days
    ahead and four days later if possible and returns a list of optional days
    :return: list
    """
    departure_from = input("Departure from: ")
    destination = input("Destination: ")
    chosen_day = cast_date(input("Chose day to fly: "))
    return_day = cast_date(input("Chose day to return: "))
    # check if date insert to fly is'nt passed
    today_date = date.today()

    if chosen_day > today_date and return_day > chosen_day and (return_day - chosen_day).days < 16:
        days_travel = (return_day - chosen_day).days
        Max_date_available = today_date + timedelta(days=16)
        optional_days = [departure_from, destination, (str(chosen_day), str(return_day))]

        # calculating end date by adding 4 days
        if (chosen_day - today_date).days < 16:
            days_before = (chosen_day - today_date).days
            if days_before >= 4:
                days_before = 4
            for i in range(1, days_before):
                new_tuple = chosen_day - timedelta(days=i)
                optional_days.append((str(new_tuple), str(new_tuple + timedelta(days=days_travel))))

        if (Max_date_available - chosen_day).days < 16:
            days_after = (Max_date_available - chosen_day).days
            if days_after >= 4:
                days_after = 4
            for i in range(1, days_after):
                new_tuple = chosen_day + timedelta(days=i)
                if new_tuple + timedelta(days=days_travel) > Max_date_available:
                    continue
                else:
                    optional_days.append((str(new_tuple), str(new_tuple + timedelta(days=days_travel))))
        return optional_days
    

In [ ]:
import requests
from datetime import timedelta, datetime, date
import time
import json
import os
import pandas as pd
from IPython.core.display import display, HTML


class forecast:
    # class forecast process api requests for an estimation the best date for a start the trip 
    
    def __init__(self):
        self.api = '9a136cd7531d4f90b4fc6d074a123b35'
 
    # convert your links to html tags
    @staticmethod
    def path_to_image_html(path):
        return '<img src="' + path + '" width="60" >'

    def avalute(self, list_coord, ls_info):
        """
        function avalutes the best t-map for user to drop markers based on place intended to sleep during trip
        :param list_coord: list of cordinates which are the lan and lon cordinates of markers using draw_on_map function
        :param ls_info: fly ticket information
        :return: best_result- dictionary 
        """
        
        flight_days = sorted(ls_info[2:])
        new_list = []
        '''
        adding and subtracting a day from the dates of the flights
        under the assumption that it takes a day to get to the starting compound and another day to a field field.
        '''
        for elem in range(len(flight_days)):
            fly_start = time.strptime(flight_days[elem][0], '%Y-%m-%d')
            hike_start = date(fly_start.tm_year, fly_start.tm_mon, fly_start.tm_mday) + timedelta(days=1)
            fly_start = str(date(fly_start.tm_year, fly_start.tm_mon, fly_start.tm_mday))

            return_fly = time.strptime(flight_days[elem][1], '%Y-%m-%d')
            hike_end = date(return_fly.tm_year, return_fly.tm_mon, return_fly.tm_mday) - timedelta(days=1)
            return_fly = str(date(return_fly.tm_year, return_fly.tm_mon, return_fly.tm_mday))

            if hike_start + timedelta(days=len(list_coord)) < hike_end:
                new_list.append((str(hike_start), (fly_start, return_fly)))
            else:
                print("User chose too many stations in between dates of the ticket")
                return

        for i in range(len(list_coord)):
            try:
                url = 'https://api.weatherbit.io/v2.0/forecast/daily?&lat={}&lon={}&key={}'.format(list_coord[0][0],
                                                                                                   list_coord[0][1],
                                                                                                   self.api)
                res = requests.get(url)
                self.dict_api = res.json()
                smart_dict = {}

                for event in range(len(new_list)):
                    if event == 0:
                        for index in range(len(self.dict_api['data'])):
                            if self.dict_api['data'][index]['datetime'] == new_list[event][0]:
                                break
                    else:
                        index += 1
                    # setting counter for rain and clouds
                    sum_raining = 0
                    sum_clouds = 0
                    for day_plus in range(len(list_coord)):
                        keys = ['pop', 'precip'] 
                        # add precent and precip values
                        sum_raining = sum_raining + sum(
                            [self.dict_api['data'][index + day_plus].get(key) for key in keys])
                        
                        # add clouds values
                        sum_clouds = sum_clouds + self.dict_api['data'][index + day_plus]['clouds']
                    smart_dict[new_list[event]] = (new_list[event][0], index, sum_raining/len(list_coord),
                                                   sum_clouds/len(list_coord), new_list[event][1]) 
                    # calculate average sum clouds and sum_rain
                    
                # dictionary is sorted by 3 keys: 
                # (first) amount of precipitation (second) clouds (last) earliest date
                best_result = sorted(smart_dict.values(), key=lambda val: (val[2], val[3], val[0]))

            except Exception as e:
                print(e)
                
        return best_result # return a גictionary

    
    def df_forecast(self, lon, lat, index):
        """
        function returns dataframe using the api of the weather forecast for the required period of time
        :param lon: longitude 
        :param lat: latitude
        :param index: center day???????????????????
        :return: first row of df and df.to_html
        """
        try:
            url = 'https://api.weatherbit.io/v2.0/forecast/daily?&lat={}&lon={}&key={}'.format(lon, lat, self.api)
            res = requests.get(url)
            self.dict_api = res.json()
            j = 0
            ls_forecast = []
            for i in range(index, index + 3): #???????????????????????
                self.dict_api['data'][i]['date'] = datetime.strptime(self.dict_api['data'][i]['datetime'],
                                                                     '%Y-%m-%d').date().strftime('%d.%m.%Y')
                self.dict_api['data'][i]['Temp'] = str(self.dict_api['data'][i]['temp']) + "\u00b0"
                self.dict_api['data'][i]['Clouds'] = str(self.dict_api['data'][i]['clouds']) + ' %'
                self.dict_api['data'][i]['Rain Chance'] = str(self.dict_api['data'][i]['pop']) + ' %'
                self.dict_api['data'][i]['Precipitation'] = str(self.dict_api['data'][i]['precip']) + ' mm/hr'

                copy_row = self.dict_api['data'][i]
                wanted_keys = ('date', 'Clouds', 'Temp', 'Rain Chance', 'Precipitation')
                ls_forecast.append(dict(zip(wanted_keys, [copy_row[k] for k in wanted_keys])))

                ls_forecast[j]['Description'] = self.dict_api['data'][i]['weather'].get('description')
                ls_forecast[j]['Status'] = 'https://www.weatherbit.io/static/img/icons/{}.png'.format(
                    self.dict_api['data'][i]['weather'].get('icon'))
                j += 1
                #     min(dict_api['data'], key=lambda x: x['clouds']+ x['pop'] + x['precip'])
                df = pd.DataFrame(ls_forecast)

                df.columns = df.columns.str.title()
                df = df.reindex(['Date', 'Temp', 'Clouds', 'Rain Chance', 'Precipitation', 'Description', 'Status'], axis=1)

                # for i in range(len(ls_forecast)):
                #     df['Description'].values[i] = df['Description'].values[i].title()

                pd.set_option('display.max_colwidth', None)
                pd.set_option('colheader_justify', 'center')
        except Exception as e:
            print(e)

        return df['Date'][0],  df.to_html(escape=False, formatters=dict(Status=self.path_to_image_html))


In [ ]:
# import
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import mysql.connector
from dateutil.parser import parse
import pandas as pd
import numpy as np
from time import sleep
from IPython.display import display


class sql_ticket:
    def __init__(self):
        # create connection with database
        DB = mysql.connector.connect(
            host='localhost',
            user='root',
            password='gdwY8RrWw8r',
            port='3306',
            database='python_sql'
        )
        self.db = DB
        self.cursor = DB.cursor()
        # open web driver
        self.chromedriver_path = "C:/Users/user/OneDrive - post.bgu.ac.il/FLY_Project/drivers/chromedriver_win32_92/chromedriver.exe"
        self.driver = webdriver.Chrome(executable_path=self.chromedriver_path)

    def isExist(self, values):
        """
        function allows us to know whether to use updating sentence or inserting sentence
        by checking if there is an instance in the db that corresponds to the primary keys
        :param values: primary keys are date and time of fly ticket
        :return: boolean (if appears)
        """
        cursor = self.db.cursor()
        flag = False
        select_query = """ SELECT * 
                           FROM flight_tickets
                           WHERE Depart_Date = %s AND Depart_Time = %s 
                           AND Return_Date = %s AND Return_Time = %s """
        try:
            cursor.execute(select_query, values)
            if len(cursor.fetchall()) != 0:  # Not empty means already exist
                flag = True

        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            self.db.rollback()

        finally:
            return flag

    def sql_to_df(self, query):
        """
        function converts sql to dataframe using pandas and changes the Durations columns from float to str
        :param query: sql query sentence
        :return: display dataframe
        """
        values = self.departure_date, self.return_date, self.flight_from+'%', '%'+self.destination # parms for query are date of fly ticket
        self.cursor.execute(query, values)
        results = self.cursor.fetchall()

        if len(results) != 0:  # if not empty convert

            df = pd.DataFrame.from_records(results, columns=[desc[0] for desc in self.cursor.description])

            df["Duration_1"] = df["Duration_1"].astype(int).astype(str) + 'h ' + round(
                (df["Duration_1"] - np.fix(df["Duration_1"])) * 100).astype(int).astype(str)

            df["Duration_2"] = df["Duration_2"].astype(int).astype(str) + 'h ' + round(
                (df["Duration_2"] - np.fix(df["Duration_2"])) * 100).astype(int).astype(str)

            return display(df)

        else:  # maens there is not match result then no df to display
            print('- No match was found')
            return

    def checkTableExists(self):
        try:
            self.cursor.execute("""
                SELECT COUNT(*)
                FROM flight_tickets 
                """)
            if len(self.cursor.fetchall()) != 0:  # means there are rows which return at fetchall
                return

        except mysql.connector.ProgrammingError as err:  # in case table dosen't exist so create
            self.cursor.execute("""
                  CREATE TABLE `python_sql`.`flight_tickets` (
                  `Depart_Date`DATE NOT NULL,
                  `Depart_Time` VARCHAR(100) NOT NULL,
                  `AirPorts_1` VARCHAR(100) NULL,
                  `Stops_1` VARCHAR(100) NULL,
                  `Duration_1` FLOAT NULL,
                  `Return_Date` DATE NOT NULL,
                  `Return_Time` VARCHAR(100) NOT NULL,
                  `AirPorts_2` VARCHAR(100) NULL,
                  `Stops_2` VARCHAR(100) NULL,
                  `Duration_2` FLOAT NULL,
                  `Price` INTEGER NULL,
                  PRIMARY KEY (`Depart_Date`, `Depart_Time`, `Return_Date`, `Return_Time`));
                """)

        except mysql.connector.Error as err:  # in case of somtiong wrong in cursor or connection
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            pass
        return

    def run_sky_scanner(self, args):
        self.driver.maximize_window()  # open tab at bigger window
        # set parameters about flight tickets
        self.flight_from = args[0]
        self.destination = args[1]
        self.departure_date = args[2]
        self.return_date = args[3]
        sleep(2)
        try:
            self.driver.get(
                "https://www.kayak.com/flights/{}-{}/{}/{}?sort=bestflight_a".format(self.flight_from, self.destination,
                                                                                     self.departure_date,
                                                                                     self.return_date))
            flag = True  # since element is displayed when page is loaded break while loop when appear
            while flag:
                try:
                    if not self.driver.find_element_by_class_name('prediction-info ').text == '':
                        flag = False
                    else:
                        sleep(2)
                except Exception as e:
                    print(e)
                    print("An exception occurred")
                    self.driver.quit()  # if something went wrong quit page

            # for clicking on load more results -first need to search for button using expected_conditions
            for j in range(3):
                try:
                    timeout = 60  # large amount of seconds for breaking expected_conditions
                    element_visible = EC.visibility_of_element_located((By.CLASS_NAME, 'resultsPaginator'))
                    WebDriverWait(self.driver, timeout).until(element_visible)
                    self.driver.find_element_by_class_name("resultsPaginator").click()
                    sleep(2)
                except Exception as e:
                    print("Timed out waiting for page to load")
                    pass

            # to get more results, click on the button and scroll through the site up and down
            all_tickets = self.driver.find_elements_by_class_name("mainInfo")  # tickets elements
            all_prices = self.driver.find_elements_by_class_name("multibook-dropdown")  # prices elements
            i = 3  # start index at 3 ignore (cheapest, best, quickest)

            self.checkTableExists()  # check for table
            while i < len(all_tickets):
                ticket = all_tickets[i].text.split('\n')  # split text
                ticket_price = all_prices[i].text

                pk_Dep_Date = parse(self.departure_date).date()
                pk_Ret_Date = parse(self.return_date).date()
                pk_Dep_Time = ticket[0]

                if ticket[2] == 'nonstop':
                    index = 0
                    AirPorts_1 = ticket[4] + ' - ' + ticket[6]
                elif ticket[2] == '1 stop':
                    index = 1
                    AirPorts_1 = ticket[4+index] + ' - ' + ticket[2+index] + ' - ' + ticket[6+index]
                elif ticket[2] == '2 stops':
                    index = 1
                    AirPorts_1 = ticket[4 + index] + ' - ' + ticket[2 + index].replace(',', ' - ') + ' - ' + ticket[6 + index]

                Duration_1 = float(ticket[3 + index].replace('h', '.').replace(' ', '').replace('m', ''))
                Stops_1 = ticket[2]
                pk_Ret_Time = ticket[7 + index]
                Stops_2 = ticket[9 + index]

                if ticket[9 + index] == 'nonstop':
                    AirPorts_2 = ticket[11 + index] + ' - ' + ticket[13 + index]
                elif ticket[9 + index] == '1 stop':
                    index += 1
                    AirPorts_2 = ticket[11 + index] + ' - ' + ticket[9 + index] + ' - ' + ticket[13 + index]
                elif ticket[9 + index] == '2 stops':
                    index += 1
                    AirPorts_2 = ticket[11 + index] + ' - ' + ticket[9 + index].replace(',', ' - ') + ' - ' + ticket[13 + index]

                Duration_2 = float(ticket[10 + index].replace('h', '.').replace(' ', '').replace('m', ''))

                pKeys = (pk_Dep_Date, pk_Dep_Time, pk_Ret_Date, pk_Ret_Time)  # primary keys
                Price = ticket_price[ticket_price.find("$") + 1 : ticket_price.find("\n")]
                if ',' in Price:
                    Price = Price.replace(',', '')

                try:
                    # if exist then update / else then insert
                    if not sql_ticket.isExist(self, pKeys):
                        # Preparing SQL query to INSERT a record into the database.
                        insert_state = """INSERT INTO flight_tickets
                                       (Depart_Date, Depart_Time, AirPorts_1, Stops_1, Duration_1,
                                       Return_Date, Return_Time, AirPorts_2, Stops_2, Duration_2,
                                       Price)
                                       VALUES (%s, %s, %s, %s, %s,
                                        %s, %s, %s, %s, %s,
                                        %s)"""

                        all_values = [pk_Dep_Date, pk_Dep_Time, AirPorts_1, Stops_1, Duration_1,
                                      pk_Ret_Date, pk_Ret_Time, AirPorts_2, Stops_2, Duration_2, Price]

                        self.cursor.execute(insert_state, tuple(all_values))
                        self.db.commit()  # commit to db

                    else:
                        # Preparing SQL query to UPDATE a record into the database.
                        update_state = """UPDATE flight_tickets
                                       SET Price = %s
                                       WHERE Depart_Date = %s AND Depart_Time = %s
                                       AND Return_Date = %s AND Return_Time = %s"""

                        up_values = [Price] + list(pKeys)  # add price val
                        self.cursor.execute(update_state, up_values)
                        self.db.commit()  # commit to db

                except mysql.connector.Error as err:
                    # self.db.rollback()  # in case there has been error do roll back
                    # print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)
                    pass
                i += 1

        except Exception as e:
            print(e)
            print("An exception occurred")
            self.driver.quit()

        try:
            # -- Cheapest : return the 4 cheapest flight
            Cheapest_query = """SELECT *
                             FROM flight_tickets
                             WHERE Depart_Date = %s AND Return_Date = %s 
                             AND AirPorts_1 LIKE %s AND AirPorts_1 LIKE %s
                             ORDER BY Price ASC LIMIT 5;"""

            print('\nCheapest_df')
            Cheapest_df = self.sql_to_df(Cheapest_query)

            # -- Quickest : return the 4 quickest flight
            Quickest_query = """SELECT *
                             FROM flight_tickets
                             WHERE Depart_Date = %s AND Return_Date = %s
                             AND AirPorts_1 LIKE %s AND AirPorts_1 LIKE %s
                             ORDER BY(Duration_1 + Duration_2) ASC LIMIT 5;"""

            print('\nQuickest_query')
            Quickest_df = self.sql_to_df(Quickest_query)

            # -- Early_Arrival : return the flight where you do not add a day
            # and the flight lands the earliest and returns to the country at the latest time
            Early_Arrival_query = """SELECT Depart_Date, Depart_Time, Return_Date, Return_Time, AirPorts_1, Duration_1, Stops_1, AirPorts_2, Duration_2, Stops_2, Price
                                    FROM (SELECT * ,CONVERT(SUBSTRING(Depart_Time, 7), TIME) as Arrival_At, CONVERT(SUBSTRING(Return_Time,1,5), TIME) as Leaving_At
                                            FROM flight_tickets
                                            WHERE Depart_Date = %s AND Return_Date = %s
                                            AND AirPorts_1 LIKE %s AND AirPorts_1 LIKE %s
                                            ORDER BY Arrival_At ASC, Leaving_At DESC LIMIT 5) as Early_Late_opt;"""
            print('\nEarly_Arrival_df')
            Early_Arrival_df = self.sql_to_df(Early_Arrival_query)

        except mysql.connector.Error as err:
            self.db.rollback()  # in case there has been error do roll back
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            pass

        finally:
            self.driver.close()
            return

In [5]:
# import
from geopy.geocoders import Nominatim
from folium import plugins
import geocoder
import geopy
import io
from PIL import Image
from IPython.core.display import display, HTML
import datetime
from datetime import timedelta, datetime, date

def draw_on_map(user_info):
    """
    function display a topo-map for user to drop markers based on place intended to sleep during trip
    :param user_info: information on fly ticket 
    :return: map
    """
    
    # setting a map using name of Air-port (not necessarily has to be port)
    geolocator = Nominatim(user_agent="application")
    location = geolocator.geocode(user_info[1])

    map_with_mini = folium.Map(location=(location.latitude, location.longitude), zoom_start=12,
                               width=1250, height=550, control_scale=True,
                               tiles='Stamen Terrain',
                               tooltip='This tooltip will appear on hover')
    
    # add mini map at the buttom of map
    minimap = plugins.MiniMap(toggle_display=True)
    map_with_mini.add_child(minimap)
    plugins.Fullscreen(position='topright').add_to(map_with_mini) # choosing map type
    #add measures 
    measure_control = plugins.MeasureControl(position='topleft',
                                             active_color='purple',
                                             completed_color='purpule',
                                             primary_length_unit='kilometers')
    map_with_mini.add_child(measure_control)
    
    # add tools and export=True exports the drawn shapes as a geojson file
    draw = plugins.Draw(filename='MyMap', export= True)
    draw.add_to(map_with_mini)
    
    # save map
    map_with_mini.save('MyMap.html')

    return map_with_mini

In [6]:
# import
import folium
from folium import plugins
from IPython.core.display import display, HTML
import json
import os
from termcolor import colored
import datetime
from datetime import timedelta, datetime, date


def trip_Map(path, ls_info, options=3, show_=False):
    """
    function loads the downloaded coordinates and processes data to find the best time window to travel
    based on the number of days and available dates from all_ticket_options function.
    
    :param path: directory where export map using draw_on_map function 
    :param ls_info: user_info using function all_ticket_options
    :param options: default 
    :param show_: show the dictionary, how the algorithm was selected the best value 
    :return: based on options return the amount of maps represent the route along with pop-ups 
     which are dataframe for the forecast each day
    """
    
    try:
        # load and open  from directory
        files = os.listdir(path)
        paths = [os.path.join(path, basename) for basename in files]
        new_file = max(paths, key=os.path.getctime).replace('\\', '/')
        with open(new_file) as f:
            data = json.load(f)
        ls_coord = []
        for i in range(len(data['features'])):
            if data['features'][0]['geometry']['type'] == 'Point':
                ls_coord.append(
                    data['features'][i]['geometry']['coordinates'][1:] + data['features'][i]['geometry']['coordinates'][
                                                                         :1])
        # creat a map usinf first coordinate
        mayMap = folium.Map(location=ls_coord[0], zoom_start=10,
                           width=1250, height=550, control_scale=True,
                            tiles='Stamen Terrain',
                            tooltip='This tooltip will appear on hover')

        # setting for a map
        minimap = plugins.MiniMap(toggle_display=True)
        mayMap.add_child(minimap)
        plugins.Fullscreen(position='topright').add_to(mayMap)

        
        # call avalute function to return dictionary arranged according to the sequence of days
        # appropriate for the trip 
        get_best_day = forecast().avalute(ls_coord, ls_info)
        if show_: # show the dictionary
            print(get_best_day)
        get_best_day = get_best_day[:options]  # get amount of result from dictionary

        print(colored("\nDisplay maps as option 1 is the rank the heights\n", 'green', attrs=['bold'])) # title
        opt_num = 1
        for opt in get_best_day:
            day_index = 1
            for i in range(len(ls_coord)):
                elem_cord = forecast().df_forecast(ls_coord[i][0], ls_coord[i][1], opt[1] + i)
                html = f"""
                    <body style="background-color:#ccccff;">
                        <h3>Option {opt_num}</h3>
                        <h2>Day {day_index} : {elem_cord[0]} </h2>
                        <p>{elem_cord[1]}</p>
                    </body>
                    """
                # add marker on the map with dataframe of forecast 
                iframe = folium.IFrame(html=html, width=540, height=360)
                popup = folium.Popup(iframe, max_width=600)
                folium.Marker(
                    location=ls_coord[i],
                    tooltip="Day {}".format(day_index),
                    popup=popup,
                    icon=folium.Icon(color='purple')).add_to(mayMap)
                day_index += 1

            mayMap.save('Map_Option_{} , {}.html'.format(opt[4][0], opt[4][1])) # save map
            result_Dict = {"Start": opt[4][0], "End": opt[4][1],  "Avg_Rain": opt[2], "Avg_Clouds": opt[3]}
            
            # print info about the sequence of days suitable for the trip
            print(colored('\nTravel info', attrs=['bold']))
            print(colored('_'*100, attrs=['bold']))
            print(colored 
                  ("Option {}\nStarts at {}\nReturn at {}\nIn average:\n - Rain chance per day is {}%\n - Coluds chance per day {}%\n"
                  .format(opt_num, result_Dict.get('Start'), result_Dict.get('End'), result_Dict.get('Avg_Rain'),
                         result_Dict.get('Avg_Clouds')),
                  attrs=['bold']))
            display(mayMap) # show map
            
            # using sql query retrieve information on tickets based on querys implement at run_sky_scanner function
            print(colored('Fly ticket info', attrs=['bold']))
            print(colored('_'*100, attrs=['bold']))
            print(colored(ls_info[:2] + [result_Dict.get('Start') ,result_Dict.get('End')], 'blue'))
            sql_ticket().run_sky_scanner(ls_info[:2] + [result_Dict.get('Start') ,result_Dict.get('End')])
            opt_num += 1
    except Exception as e:
        print(e)

    finally:
        return

In [9]:
def run_app():
#     print("Departure from: TLV\n"
#           "Destination: valbona\n"
#           "Chose day to fly: 24.09.2021\n"
#           "Chose day to return: 03.10.2021\n")
    
    user_info = all_ticket_options()

    return draw_on_map(user_info), user_info

results = run_app()


Departure from: TLV
Destination: CDG
Chose day to fly: 20.11.2021
Chose day to return: 27.11.2021


In [10]:
results[0]

In [12]:
trip_Map('C:/Users/user/Downloads', results[1], show_=True)


[('2021-11-23', 8, 0.0, 37.25, ('2021-11-22', '2021-11-29')), ('2021-11-18', 3, 11.34375, 76.5, ('2021-11-17', '2021-11-24')), ('2021-11-24', 9, 16.734375, 58.25, ('2021-11-23', '2021-11-30')), ('2021-11-22', 7, 21.0625, 33.0, ('2021-11-21', '2021-11-28')), ('2021-11-21', 6, 27.390625, 39.25, ('2021-11-20', '2021-11-27')), ('2021-11-20', 5, 32.40625, 47.5, ('2021-11-19', '2021-11-26')), ('2021-11-19', 4, 32.40625, 66.0, ('2021-11-18', '2021-11-25'))]

Display maps as option 1 is the rank the heights


Travel info
____________________________________________________________________________________________________
Option 1
Starts at 2021-11-22
Return at 2021-11-29
In average:
 - Rain chance per day is 0.0%
 - Coluds chance per day 37.25%



Fly ticket info
____________________________________________________________________________________________________
['TLV', 'CDG', '2021-11-22', '2021-11-29']

Cheapest_df


,Depart_Date,Depart_Time,AirPorts_1,Stops_1,Duration_1,Return_Date,Return_Time,AirPorts_2,Stops_2,Duration_2,Price
0,2021-11-22,8:30 pm – 8:05 am +1,TLV - MXP - CDG,1 stop,12h 35,2021-11-29,11:00 am – 7:30 pm,CDG - MXP - TLV,1 stop,7h 30,209
1,2021-11-22,8:30 pm – 8:05 am +1,TLV - MXP - CDG,1 stop,12h 35,2021-11-29,7:30 am – 4:10 pm,CDG - BGY - TLV,1 stop,7h 40,230
2,2021-11-22,5:30 pm – 8:30 am +1,TLV - BGY-LIN - CDG,1 stop,16h 0,2021-11-29,12:10 pm – 5:45 pm,CDG - TLV,nonstop,4h 35,254
3,2021-11-22,5:30 pm – 8:05 am +1,TLV - BGY-MXP - CDG,1 stop,15h 35,2021-11-29,12:10 pm – 5:45 pm,CDG - TLV,nonstop,4h 35,260
4,2021-11-22,10:55 am – 6:10 pm,TLV - KRK - CDG,1 stop,8h 15,2021-11-29,12:10 pm – 5:45 pm,CDG - TLV,nonstop,4h 35,266



Quickest_query


,Depart_Date,Depart_Time,AirPorts_1,Stops_1,Duration_1,Return_Date,Return_Time,AirPorts_2,Stops_2,Duration_2,Price
0,2021-11-22,3:55 pm – 7:35 pm,TLV - CDG,nonstop,4h 40,2021-11-29,8:55 am – 2:05 pm,CDG - TLV,nonstop,4h 10,578
1,2021-11-22,4:50 pm – 8:55 pm,TLV - CDG,nonstop,5h 5,2021-11-29,2:45 pm – 8:00 pm,CDG - TLV,nonstop,4h 15,739
2,2021-11-22,4:50 pm – 8:55 pm,TLV - CDG,nonstop,5h 5,2021-11-29,10:30 pm – 3:55 am +1,CDG - TLV,nonstop,4h 25,740
3,2021-11-22,6:50 pm – 11:10 pm,TLV - CDG,nonstop,5h 20,2021-11-29,2:45 pm – 8:00 pm,CDG - TLV,nonstop,4h 15,533
4,2021-11-22,4:50 pm – 8:55 pm,TLV - CDG,nonstop,5h 5,2021-11-29,12:10 pm – 5:45 pm,CDG - TLV,nonstop,4h 35,560


...

,Depart_Date,Depart_Time,AirPorts_1,Stops_1,Duration_1,Return_Date,Return_Time,AirPorts_2,Stops_2,Duration_2,Price
0,2021-11-23,5:30 am – 12:10 pm,TLV - KBP - CDG,1 stop,7h 40,2021-11-30,1:20 pm – 11:10 pm,CDG - KBP - TLV,1 stop,8h 50,321
1,2021-11-23,5:30 am – 12:10 pm,TLV - KBP - CDG,1 stop,7h 40,2021-11-30,6:45 pm – 3:10 am +1,CDG - FCO - TLV,1 stop,7h 25,322
2,2021-11-23,7:35 am – 2:10 pm,TLV - ZRH - CDG,1 stop,7h 35,2021-11-30,8:10 pm – 3:30 am +1,CDG - ZRH - TLV,1 stop,6h 20,327
3,2021-11-23,7:35 am – 7:15 pm,TLV - ZRH - CDG,1 stop,12h 40,2021-11-30,8:10 pm – 3:30 am +1,CDG - ZRH - TLV,1 stop,6h 20,327
4,2021-11-23,7:35 am – 2:10 pm,TLV - ZRH - CDG,1 stop,7h 35,2021-11-30,3:15 pm – 3:30 am +1,CDG - ZRH - TLV,1 stop,11h 15,327



Quickest_query


,Depart_Date,Depart_Time,AirPorts_1,Stops_1,Duration_1,Return_Date,Return_Time,AirPorts_2,Stops_2,Duration_2,Price
0,2021-11-23,9:00 am – 1:00 pm,TLV - CDG,nonstop,5h 0,2021-11-30,2:45 pm – 8:00 pm,CDG - TLV,nonstop,4h 15,715
1,2021-11-23,4:50 pm – 8:55 pm,TLV - CDG,nonstop,5h 5,2021-11-30,2:45 pm – 8:00 pm,CDG - TLV,nonstop,4h 15,636
2,2021-11-23,9:00 am – 1:00 pm,TLV - CDG,nonstop,5h 0,2021-11-30,10:30 pm – 3:55 am +1,CDG - TLV,nonstop,4h 25,685
3,2021-11-23,4:50 pm – 8:55 pm,TLV - CDG,nonstop,5h 5,2021-11-30,10:30 pm – 3:55 am +1,CDG - TLV,nonstop,4h 25,607
4,2021-11-23,3:55 pm – 7:35 pm,TLV - CDG,nonstop,4h 40,2021-11-30,6:40 pm – 2:20 am +1,CDG - AMS - TLV,1 stop,6h 40,586



Early_Arrival_df


,Depart_Date,Depart_Time,Return_Date,Return_Time,AirPorts_1,Duration_1,Stops_1,AirPorts_2,Duration_2,Stops_2,Price
0,2021-11-23,3:55 pm – 7:35 pm,2021-11-30,12:40 pm – 2:20 am +1,TLV - CDG,4h 40,nonstop,CDG - AMS - TLV,12h 40,1 stop,561
1,2021-11-23,5:30 am – 12:10 pm,2021-11-30,12:00 pm – 10:30 pm,TLV - KBP - CDG,7h 40,1 stop,CDG - OTP - TLV,9h 30,1 stop,402
2,2021-11-23,10:00 pm – 11:00 am +1,2021-11-30,10:30 pm – 3:55 am +1,TLV - OTP - CDG,14h 0,1 stop,CDG - TLV,4h 25,nonstop,373
3,2021-11-23,11:30 pm – 11:00 am +1,2021-11-30,10:30 pm – 3:55 am +1,TLV - OTP - CDG,12h 30,1 stop,CDG - TLV,4h 25,nonstop,368
4,2021-11-23,4:50 pm – 8:55 pm,2021-11-30,10:30 pm – 3:55 am +1,TLV - CDG,5h 5,nonstop,CDG - TLV,4h 25,nonstop,607
